# Code source de l'algorithme de prédiction de gisements de matériaux bruts 
Dans le cadre de mon projet de fin d'études en architecture, soutenu en Juillet 2021

## Chargement des données

Les données d'entraînement proviennent du jeu de données agrégé du [Chapitre 3](CHAPITRE_3.ipynb)

In [2]:
import geopandas as gpd
import tensorflow as tf
import autokeras as ak
import numpy as np
from PIL import Image

DATA_AGG_PATH = os.path.join("_DATA_agg","BATIMENTS_GRAND_PARIS.gpkg")

buildings = gpd.read_file(DATA_AGG_PATH)

#### Chargement des images

In [3]:
BUILDINGS_IMGS_DIR = os.path.join("_DATA_agg","BATIMENTS_GRAND_PARIS_imgs")

def load_img(building_id):
    path = os.path.join(BUILDINGS_IMGS_DIR,f"{building_id}.png")
    if os.path.exists(path):
        img = Image.open(path)
        return np.asarray(img)
    return None

# Chargement des photographies et suppression des bâtiments sans photographie
buildings["img"] = buildings["id"].apply(load_img)
buildings = buildings.dropna()

#### Calcul du volume de matériau structurel de façade

In [5]:
def get_facade_material_volume(building):
    # Calcul de la surface de façade
    S_facade = building["surface_parois"] - building["surface_vitrage"]
    # Calcul de l'épaisseur de la façade
    ep = building["surface_emprise"] - (building["surface_habitable"]/building["nb_niveaux"])
    # Calcul du volume
    building["volume_materiau_facade"] = S_facade*ep
    return building

# Calcul du volume de matériaux de facade pour chaque bâtiment
buildings = buildings.apply(get_facade_material_volume, axis=1)

## Entraînement du modèle

|Features à fournir|type|Features à prédire|type|
|---|---|---|---|
|img |*image*|materiau_facade|*numérique*|
|surface_emprise |*numérique*|volume_materiau_facade|*catégorique*|
|hauteur_mediane |*numérique*|||
|typologie_bati_apur |*catégorique*|||
|periode_construction| *catégorique*|||



#### Sélection des features pour l'entraînement

In [ ]:

# Sélection des données pour l'entraînement
# X : photographie aérienne, surface d'emprise, hauteur, typologie du bâtiment et période de construction
X_img = np.stack(buildings["img"].to_list())
X_features = buildings[["surface_emprise","hauteur_mediane","typologie_bati_apur","periode_construction"]].values

# y : matériau de façade et son volume
y_mat = buildings["materiau_facade"].to_numpy()
y_vol = buildings["volume_materiau_facade"].to_numpy()

# Initialisation du modèle
amdl = ak.AutoModel(
    inputs=[
        ak.ImageInput(), # Photographie
        ak.StructuredDataInput() # Données sur la bâtiment
        ],
    outputs=[
        ak.ClassificationHead(loss='categorical_crossentropy', metrics=['accuracy']), # Matériau structurel de façade
        ak.RegressionHead(metrics=['mae']) # Volume
        ],
    overwrite=True,
    max_trials=5,
    directory=os.path.join("_MDL","mdl_prediction_volume_materiaux_facade_temp")
    )

# Entraînement 
amdl.fit(
    [X_img, X_features],
    [y_mat, y_vol],
    validation_split=0.15,
    epochs=10
    )

# Sauvegarde du meilleur modèle
final_mdl = amdl.export_model()
final_mdl_path = os.path.join("_MDL","mdl_prediction_volume_materiaux_facade")
final_mdl.save(final_mdl_path, save_format="tf")

## Prédiction sur l'existant

Données récoltées manuellement sur site sur 210 bâtiments du quartier Ivry Port:

* Photographie aérienne
* Surface de l'emprise
* Hauteur
* Typologie
* Période de construction



In [8]:
import pandas as pd
from tensorflow.keras.models import load_model

EXISTING_BUILDINGS_DIR = os.path.join("_DATA","PFE_EXISTANT")
EXISTING_BUILDINGS_DATA = os.path.join(EXISTING_BUILDINGS_DIR,"existant.csv")
EXISTING_BUILDINGS_IMGS_DIR = os.path.join(EXISTING_BUILDINGS_DIR,"existant_imgs")
MDL_PATH = os.path.join("_MDL","mdl_prediction_volume_materiaux_facade")

ex_buildings = pd.read_csv(EXISTING_BUILDINGS_DATA)

def load_img(building_id):
    path = os.path.join(EXISTING_BUILDINGS_IMGS_DIR,f"{building_id}.png")
    if os.path.exists(path):
        img = Image.open(path)
        return np.asarray(img)
    return None

# Chargement des photographies 
ex_buildings["img"] = ex_buildings["id"].apply(load_img)

# Sélection des features pour la prédiction
ex_X_img = np.stack(ex_buildings["img"].to_list())
ex_X_features = ex_buildings[["surface_emprise","hauteur_mediane","typologie_bati_apur","periode_construction"]].values

In [21]:
# Chargement du modèle et prédiction sur l'existant
mdl = load_model(MDL_PATH, custom_objects=ak.CUSTOM_OBJECTS)
ex_y = mdl.predict([ex_X_img,ex_X_features])

ex_buildings["materiau_facade"] = ex_y[0].tolist()
ex_buildings["volume_materiau_facade"] = ex_y[1].tolist()

# Rappel 
MATERIALS = {
    1 : "Pierre",
    2 : "Meulière",
    3 : "Béton",
    4 : "Briques",
    5 : "Aggloméré",
    6 : "Bois"
}

# Obtention de la catégorie correspondant au pourcentage le plus élevé (catégorie la plus "probable")
ex_buildings["materiau_facade"] = ex_buildings["materiau_facade"].apply(lambda x: MATERIALS[x.index(max(x))+1])
# Extraction du volume hors d'une liste
ex_buildings["volume_materiau_facade"] = ex_buildings["volume_materiau_facade"].apply(lambda x: x[0])

# Résultats
from IPython.core.display import HTML

sample_ex_buildings = ex_buildings.sample(3)
sample_ex_buildings["img"] = sample_ex_buildings["id"].apply(lambda x: '<img src="' + os.path.join(EXISTING_BUILDINGS_IMGS_DIR, f"{x}.png") + '" width="60">')
HTML(sample_ex_buildings[["img", "materiau_facade", "volume_materiau_facade"]].to_html(escape=False))

7/7 [==============================] - 3s 299ms/step


,img,materiau_facade,volume_materiau_facade
193,,Briques,253.513184
55,,Briques,136.674667
73,,Briques,1150.777710
